# Amazon SageMaker Multi-Model Endpoints using XGBoost
With [Amazon SageMaker multi-model endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html), customers can create an endpoint that seamlessly hosts up to thousands of models. These endpoints are well suited to use cases where any one of a large number of models, which can be served from a common inference container, needs to be invokable on-demand and where it is acceptable for infrequently invoked models to incur some additional latency. For applications which require consistently low inference latency, a traditional endpoint is still the best choice.

At a high level, Amazon SageMaker manages the loading and unloading of models for a multi-model endpoint, as they are needed. When an invocation request is made for a particular model, Amazon SageMaker routes the request to an instance assigned to that model, downloads the model artifacts from S3 onto that instance, and initiates loading of the model into the memory of the container. As soon as the loading is complete, Amazon SageMaker performs the requested invocation and returns the result. If the model is already loaded in memory on the selected instance, the downloading and loading steps are skipped and the invocation is performed immediately.

To demonstrate how multi-model endpoints are created and used, this notebook provides an example using a set of XGBoost models that each predict housing prices for a single location. This domain is used as a simple example to easily experiment with multi-model endpoints.

The Amazon SageMaker multi-model endpoint capability is designed to work across all machine learning frameworks and algorithms including those where you bring your own container.

### Contents

1. [Build and register an XGBoost container that can serve multiple models](#Build-and-register-an-XGBoost-container-that-can-serve-multiple-models)
1. [Generate synthetic data for housing models](#Generate-synthetic-data-for-housing-models)
1. [Train multiple house value prediction models](#Train-multiple-house-value-prediction-models)
1. [Import models into hosting](#Import-models-into-hosting)
  1. [Deploy model artifacts to be found by the endpoint](#Deploy-model-artifacts-to-be-found-by-the-endpoint)
  1. [Create the Amazon SageMaker model entity](#Create-the-Amazon-SageMaker-model-entity)
  1. [Create the multi-model endpoint](#Create-the-multi-model-endpoint)
1. [Exercise the multi-model endpoint](#Exercise-the-multi-model-endpoint)
  1. [Dynamically deploy another model](#Dynamically-deploy-another-model)
  1. [Invoke the newly deployed model](#Invoke-the-newly-deployed-model)
  1. [Updating a model](#Updating-a-model)
1. [Clean up](#Clean-up)

## Build and register an XGBoost container that can serve multiple models

In [1]:
!pip install -qU awscli boto3 sagemaker

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


For the inference container to serve multiple models in a multi-model endpoint, it must implement [additional APIs](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html) in order to load, list, get, unload and invoke specific models.

The ['mme' branch of the SageMaker XGBoost Container repository](https://github.com/aws/sagemaker-xgboost-container/tree/mme) is an example implementation on how to adapt SageMaker's XGBoost framework container to use [Multi Model Server](https://github.com/awslabs/multi-model-server), a framework that provides an HTTP frontend that implements the additional container APIs required by multi-model endpoints, and also provides a pluggable backend handler for serving models using a custom framework, in this case the XGBoost framework.

Using this branch, below we will build an XGBoost container that fulfills all of the multi-model endpoint container requirements, and then upload that image to Amazon Elastic Container Registry (ECR). Because uploading the image to ECR may create a new ECR repository, this notebook requires permissions in addition to the regular SageMakerFullAccess permissions. The easiest way to add these permissions is simply to add the managed policy AmazonEC2ContainerRegistryFullAccess to the role that you used to start your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

In [2]:
ALGORITHM_NAME = 'multi-model-xgboost'

In [3]:
%%sh -s $ALGORITHM_NAME

algorithm_name=$1

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration
region=$(aws configure get region)

ecr_image="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email --registry-ids ${account})

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# First clear out any prior version of the cloned repo
rm -rf sagemaker-xgboost-container/

# Clone the xgboost container repo
git clone --single-branch --branch mme https://github.com/aws/sagemaker-xgboost-container.git
cd sagemaker-xgboost-container/

# Build the "base" container image that encompasses the installation of the
# XGBoost framework and all of the dependencies needed.
docker build -q -t xgboost-container-base:0.90-2-cpu-py3 -f docker/0.90-2/base/Dockerfile.cpu .

# Create the SageMaker XGBoost Container Python package.
python setup.py bdist_wheel --universal

# Build the "final" container image that encompasses the installation of the
# code that implements the SageMaker multi-model container requirements.
docker build -q -t ${algorithm_name} -f docker/0.90-2/final/Dockerfile.cpu .

docker tag ${algorithm_name} ${ecr_image}

docker push ${ecr_image}

Login Succeeded
sha256:bd8f0a3585b13d93f2d5e77957c0bcab06350bcac32ea823de000cd7cfb85b05
running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/distributed.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/training.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/checkpointing.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/data_utils.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/handler_service.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/encoder.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/serving.py -> build/lib/sagemaker_xgboost_container
copying src/sagemaker_xgboost_container/__init__.py -> build/lib/sagemaker_xgboost_container
creating build/lib/sa

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Cloning into 'sagemaker-xgboost-container'...


## Generate synthetic data for housing models

In [4]:
import numpy as np
import pandas as pd
import json
import datetime
import time
from time import gmtime, strftime
import matplotlib.pyplot as plt

In [5]:
NUM_HOUSES_PER_LOCATION = 1000
LOCATIONS  = ['NewYork_NY',    'LosAngeles_CA',   'Chicago_IL',    'Houston_TX',   'Dallas_TX',
              'Phoenix_AZ',    'Philadelphia_PA', 'SanAntonio_TX', 'SanDiego_CA',  'SanFrancisco_CA']
PARALLEL_TRAINING_JOBS = 4 # len(LOCATIONS) if your account limits can handle it
MAX_YEAR = 2019

In [6]:
def gen_price(house):
    _base_price = int(house['SQUARE_FEET'] * 150)
    _price = int(_base_price + (10000 * house['NUM_BEDROOMS']) + \
                               (15000 * house['NUM_BATHROOMS']) + \
                               (15000 * house['LOT_ACRES']) + \
                               (15000 * house['GARAGE_SPACES']) - \
                               (5000 * (MAX_YEAR - house['YEAR_BUILT'])))
    return _price

In [7]:
def gen_random_house():
    _house = {'SQUARE_FEET':   int(np.random.normal(3000, 750)),
              'NUM_BEDROOMS':  np.random.randint(2, 7),
              'NUM_BATHROOMS': np.random.randint(2, 7) / 2,
              'LOT_ACRES':     round(np.random.normal(1.0, 0.25), 2),
              'GARAGE_SPACES': np.random.randint(0, 4),
              'YEAR_BUILT':    min(MAX_YEAR, int(np.random.normal(1995, 10)))}
    _price = gen_price(_house)
    return [_price, _house['YEAR_BUILT'],   _house['SQUARE_FEET'], 
                    _house['NUM_BEDROOMS'], _house['NUM_BATHROOMS'], 
                    _house['LOT_ACRES'],    _house['GARAGE_SPACES']]

In [8]:
def gen_houses(num_houses):
    _house_list = []
    for i in range(num_houses):
        _house_list.append(gen_random_house())
    _df = pd.DataFrame(_house_list, 
                       columns=['PRICE',        'YEAR_BUILT',    'SQUARE_FEET',  'NUM_BEDROOMS',
                                'NUM_BATHROOMS','LOT_ACRES',     'GARAGE_SPACES'])
    return _df

## Train multiple house value prediction models

In [9]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
REGION     = boto3.Session().region_name
BUCKET     = sagemaker_session.default_bucket()

MULTI_MODEL_XGBOOST_IMAGE = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(ACCOUNT_ID, REGION, 
                                                                           ALGORITHM_NAME)

DATA_PREFIX            = 'DEMO_MME_XGBOOST'
HOUSING_MODEL_NAME     = 'housing'
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

### Split a given dataset into train, validation, and test

In [10]:
from sklearn.model_selection import train_test_split
SEED = 7
SPLIT_RATIOS = [0.6, 0.3, 0.1]

def split_data(df):
    # split data into train and test sets
    seed      = SEED
    val_size  = SPLIT_RATIOS[1]
    test_size = SPLIT_RATIOS[2]
    
    num_samples = df.shape[0]
    X1 = df.values[:num_samples, 1:] # keep only the features, skip the target, all rows
    Y1 = df.values[:num_samples, :1] # keep only the target, all rows

    # Use split ratios to divide up into train/val/test
    X_train, X_val, y_train, y_val = \
        train_test_split(X1, Y1, test_size=(test_size + val_size), random_state=seed)
    # Of the remaining non-training samples, give proper ratio to validation and to test
    X_test, X_test, y_test, y_test = \
        train_test_split(X_val, y_val, test_size=(test_size / (test_size + val_size)), 
                         random_state=seed)
    # reassemble the datasets with target in first column and features after that
    _train = np.concatenate([y_train, X_train], axis=1)
    _val   = np.concatenate([y_val,   X_val],   axis=1)
    _test  = np.concatenate([y_test,  X_test],  axis=1)

    return _train, _val, _test

### Launch a single training job for a given housing location
There is nothing specific to multi-model endpoints in terms of the models it will host. They are trained in the same way as all other SageMaker models. Here we are using the XGBoost estimator and not waiting for the job to complete.

In [11]:
def launch_training_job(location):
    # clear out old versions of the data
    s3_bucket = s3.Bucket(BUCKET)
    full_input_prefix = '{}/model_prep/{}'.format(DATA_PREFIX, location)
    s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

    # upload the entire set of data for all three channels
    local_folder = 'data/{}'.format(location)
    inputs = sagemaker_session.upload_data(path=local_folder, key_prefix=full_input_prefix)
    print('Training data uploaded: {}'.format(inputs))
    
    _job = 'xgb-{}'.format(location.replace('_', '-'))
    full_output_prefix = '{}/model_artifacts/{}'.format(DATA_PREFIX, location)
    s3_output_path = 's3://{}/{}'.format(BUCKET, full_output_prefix)

    xgb = sagemaker.estimator.Estimator(MULTI_MODEL_XGBOOST_IMAGE, role, 
                                        train_instance_count=1, train_instance_type=TRAIN_INSTANCE_TYPE,
                                        output_path=s3_output_path, base_job_name=_job,
                                        sagemaker_session=sagemaker_session)
    xgb.set_hyperparameters(max_depth=5, eta=0.2, gamma=4, min_child_weight=6, subsample=0.8, silent=0, 
                            early_stopping_rounds=5, objective='reg:linear', num_round=25) 
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    train_input = sagemaker.s3_input(s3_data=inputs+'/train', 
                                     distribution=DISTRIBUTION_MODE, content_type='csv')
    val_input   = sagemaker.s3_input(s3_data=inputs+'/val', 
                                     distribution=DISTRIBUTION_MODE, content_type='csv')
    remote_inputs = {'train': train_input, 'validation': val_input}

    xgb.fit(remote_inputs, wait=False)
    
    return xgb.latest_training_job.name

### Kick off a model training job for each housing location

In [12]:
def save_data_locally(location, train, val, test):
    os.makedirs('data/{}/train'.format(location))
    np.savetxt( 'data/{0}/train/{0}_train.csv'.format(location), train, delimiter=',', fmt='%.2f')
    
    os.makedirs('data/{}/val'.format(location))
    np.savetxt( 'data/{0}/val/{0}_val.csv'.format(location),     val, delimiter=',', fmt='%.2f')
    
    os.makedirs('data/{}/test'.format(location))
    np.savetxt( 'data/{0}/test/{0}_test.csv'.format(location),   test, delimiter=',', fmt='%.2f')

In [13]:
import shutil
import os

training_jobs = []

shutil.rmtree('data', ignore_errors=True)

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    _houses = gen_houses(NUM_HOUSES_PER_LOCATION)
    _train, _val, _test = split_data(_houses)
    save_data_locally(loc, _train, _val, _test)
    _job = launch_training_job(loc)
    training_jobs.append(_job)
print('{} training jobs launched: {}'.format(len(training_jobs), training_jobs))

Training data uploaded: s3://sagemaker-us-east-2-811346863928/DEMO_MME_XGBOOST/model_prep/NewYork_NY
Training data uploaded: s3://sagemaker-us-east-2-811346863928/DEMO_MME_XGBOOST/model_prep/LosAngeles_CA
Training data uploaded: s3://sagemaker-us-east-2-811346863928/DEMO_MME_XGBOOST/model_prep/Chicago_IL
Training data uploaded: s3://sagemaker-us-east-2-811346863928/DEMO_MME_XGBOOST/model_prep/Houston_TX
4 training jobs launched: ['xgb-NewYork-NY-2020-01-22-06-41-05-962', 'xgb-LosAngeles-CA-2020-01-22-06-41-06-341', 'xgb-Chicago-IL-2020-01-22-06-41-08-929', 'xgb-Houston-TX-2020-01-22-06-41-10-317']


### Wait for all model training to finish

In [14]:
def wait_for_training_job_to_complete(job_name):
    print('Waiting for job {} to complete...'.format(job_name))
    resp = sm_client.describe_training_job(TrainingJobName=job_name)
    status = resp['TrainingJobStatus']
    while status=='InProgress':
        time.sleep(60)
        resp = sm_client.describe_training_job(TrainingJobName=job_name)
        status = resp['TrainingJobStatus']
        if status == 'InProgress':
            print('{} job status: {}'.format(job_name, status))
    print('DONE. Status for {} is {}\n'.format(job_name, status))

In [15]:
# wait for the jobs to finish
for j in training_jobs:
    wait_for_training_job_to_complete(j)

Waiting for job xgb-NewYork-NY-2020-01-22-06-41-05-962 to complete...
xgb-NewYork-NY-2020-01-22-06-41-05-962 job status: InProgress
xgb-NewYork-NY-2020-01-22-06-41-05-962 job status: InProgress
xgb-NewYork-NY-2020-01-22-06-41-05-962 job status: InProgress
DONE. Status for xgb-NewYork-NY-2020-01-22-06-41-05-962 is Completed

Waiting for job xgb-LosAngeles-CA-2020-01-22-06-41-06-341 to complete...
DONE. Status for xgb-LosAngeles-CA-2020-01-22-06-41-06-341 is Completed

Waiting for job xgb-Chicago-IL-2020-01-22-06-41-08-929 to complete...
DONE. Status for xgb-Chicago-IL-2020-01-22-06-41-08-929 is Completed

Waiting for job xgb-Houston-TX-2020-01-22-06-41-10-317 to complete...
DONE. Status for xgb-Houston-TX-2020-01-22-06-41-10-317 is Completed



## Import models into hosting
A big difference for multi-model endpoints is that when creating the Model entity, the container's `ModelDataUrl` is the S3 prefix where the model artifacts that are invokable by the endpoint are located. The rest of the S3 path will be specified when actually invoking the model. Remember to close the location with a trailing slash.

The `Mode` of container is specified as `MultiModel` to signify that the container will host multiple models.

### Deploy model artifacts to be found by the endpoint
As described above, the multi-model endpoint is configured to find its model artifacts in a specific location in S3. For each trained model, we make a copy of its model artifacts into that location.

In our example, we are storing all the models within a single folder. The implementation of multi-model endpoints is flexible enough to permit an arbitrary folder structure. For a set of housing models for example, you could have a top level folder for each region, and the model artifacts would be copied to those regional folders. The target model referenced when invoking such a model would include the folder path. For example, `northeast/Boston_MA.tar.gz`.

In [16]:
import re
def parse_model_artifacts(model_data_url):
    # extract the s3 key from the full url to the model artifacts
    _s3_key = model_data_url.split('s3://{}/'.format(BUCKET))[1]
    # get the part of the key that identifies the model within the model artifacts folder
    _model_name_plus = _s3_key[_s3_key.find('model_artifacts') + len('model_artifacts') + 1:]
    # finally, get the unique model name (e.g., "NewYork_NY")
    _model_name = re.findall('^(.*?)/', _model_name_plus)[0]
    return _s3_key, _model_name 

In [17]:
# make a copy of the model artifacts from the original output of the training job to the place in
# s3 where the multi model endpoint will dynamically load individual models
def deploy_artifacts_to_mme(job_name):
    _resp = sm_client.describe_training_job(TrainingJobName=job_name)
    _source_s3_key, _model_name = parse_model_artifacts(_resp['ModelArtifacts']['S3ModelArtifacts'])
    _copy_source = {'Bucket': BUCKET, 'Key': _source_s3_key}
    _key = '{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, _model_name)
    
    print('Copying {} model\n   from: {}\n     to: {}...'.format(_model_name, _source_s3_key, _key))
    s3_client.copy_object(Bucket=BUCKET, CopySource=_copy_source, Key=_key)
    return _key

Note that we are purposely *not* copying the first model. This will be copied later in the notebook to demonstrate how to dynamically add new models to an already running endpoint.

In [18]:
# First, clear out old versions of the model artifacts from previous runs of this notebook
s3 = boto3.resource('s3')
s3_bucket = s3.Bucket(BUCKET)
full_input_prefix = '{}/multi_model_artifacts'.format(DATA_PREFIX)
print('Removing old model artifacts from {}'.format(full_input_prefix))
s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

Removing old model artifacts from DEMO_MME_XGBOOST/multi_model_artifacts


[]

In [19]:
# copy every model except the first one
for job in training_jobs[1:]:
    deploy_artifacts_to_mme(job)

Copying LosAngeles_CA model
   from: DEMO_MME_XGBOOST/model_artifacts/LosAngeles_CA/xgb-LosAngeles-CA-2020-01-22-06-41-06-341/output/model.tar.gz
     to: DEMO_MME_XGBOOST/multi_model_artifacts/LosAngeles_CA.tar.gz...
Copying Chicago_IL model
   from: DEMO_MME_XGBOOST/model_artifacts/Chicago_IL/xgb-Chicago-IL-2020-01-22-06-41-08-929/output/model.tar.gz
     to: DEMO_MME_XGBOOST/multi_model_artifacts/Chicago_IL.tar.gz...
Copying Houston_TX model
   from: DEMO_MME_XGBOOST/model_artifacts/Houston_TX/xgb-Houston-TX-2020-01-22-06-41-10-317/output/model.tar.gz
     to: DEMO_MME_XGBOOST/multi_model_artifacts/Houston_TX.tar.gz...


### Create the Amazon SageMaker model entity
Here we use `boto3` to create the model entity. Instead of describing a single model, it will indicate the use of multi-model semantics and will identify the source location of all specific model artifacts.

In [20]:
def create_multi_model_entity(multi_model_name, role):
    # establish the place in S3 from which the endpoint will pull individual models
    _model_url  = 's3://{}/{}/{}/'.format(BUCKET, DATA_PREFIX, MULTI_MODEL_ARTIFACTS)
    _container = {
        'Image':        MULTI_MODEL_XGBOOST_IMAGE,
        'ModelDataUrl': _model_url,
        'Mode':         'MultiModel'
    }
    create_model_response = sm_client.create_model(
        ModelName = multi_model_name,
        ExecutionRoleArn = role,
        Containers = [_container])
    
    return _model_url

In [21]:
multi_model_name = '{}-{}'.format(HOUSING_MODEL_NAME, strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
model_url = create_multi_model_entity(multi_model_name, role)
print('Multi model name: {}'.format(multi_model_name))

Multi model name: housing-2020-01-22-06-46-12


### Create the multi-model endpoint
There is nothing special about the SageMaker endpoint config for a multi-model endpoint. You need to consider the appropriate instance type and number of instances for the projected prediction workload. The number and size of the individual models will drive memory requirements.

Once the endpoint config is in place, the endpoint creation is straightforward.

In [22]:
endpoint_config_name = multi_model_name
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': ENDPOINT_INSTANCE_TYPE,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': multi_model_name,
        'VariantName': 'AllTraffic'}])

endpoint_name = multi_model_name
print('Endpoint name: ' + endpoint_name)

Endpoint config name: housing-2020-01-22-06-46-12
Endpoint name: housing-2020-01-22-06-46-12


In [23]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

Endpoint Arn: arn:aws:sagemaker:us-east-2:811346863928:endpoint/housing-2020-01-22-06-46-12


In [24]:
print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Waiting for housing-2020-01-22-06-46-12 endpoint to be in service...


## Exercise the multi-model endpoint

### Invoke multiple individual models hosted behind a single endpoint
Here we iterate through a set of housing predictions, choosing the specific location-based housing model at random. Notice the cold start price paid for the first invocation of any given model. Subsequent invocations of the same model take advantage of the model already being loaded into memory.

In [25]:
def predict_one_house_value(features, model_name):
    print('Using model {} to predict price of this house: {}'.format(full_model_name,
                                                                     features))
    body = ','.join(map(str, features)) + '\n'
    start_time = time.time()

    response = runtime_sm_client.invoke_endpoint(
                        EndpointName=endpoint_name,
                        ContentType='text/csv',
                        TargetModel=full_model_name,
                        Body=body)
    predicted_value = json.loads(response['Body'].read())[0]

    duration = time.time() - start_time
    
    print('${:,.2f}, took {:,d} ms\n'.format(predicted_value, int(duration * 1000)))

In [26]:
print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls --human-readable --summarize $model_url

Here are the models that the endpoint has at its disposal:
2020-01-22 06:46:13   11.3 KiB Chicago_IL.tar.gz
2020-01-22 06:46:13   11.0 KiB Houston_TX.tar.gz
2020-01-22 06:46:13   11.7 KiB LosAngeles_CA.tar.gz

Total Objects: 3
   Total Size: 34.0 KiB


In [27]:
# iterate through invocations with random inputs against a random model showing results and latency
for i in range(10):
    model_name = LOCATIONS[np.random.randint(1, len(LOCATIONS[:PARALLEL_TRAINING_JOBS]))]
    full_model_name = '{}.tar.gz'.format(model_name)
    predict_one_house_value(gen_random_house()[1:], full_model_name)

Using model LosAngeles_CA.tar.gz to predict price of this house: [1986, 2025, 6, 1.5, 0.99, 3]
$266,988.03, took 1,245 ms

Using model Houston_TX.tar.gz to predict price of this house: [2019, 3954, 2, 2.0, 0.79, 1]
$652,378.88, took 790 ms

Using model Chicago_IL.tar.gz to predict price of this house: [1988, 2468, 4, 2.5, 0.85, 1]
$329,907.84, took 811 ms

Using model Chicago_IL.tar.gz to predict price of this house: [1983, 2930, 6, 3.0, 1.16, 0]
$371,692.84, took 22 ms

Using model Houston_TX.tar.gz to predict price of this house: [1975, 2328, 5, 2.0, 1.01, 1]
$231,938.58, took 17 ms

Using model Houston_TX.tar.gz to predict price of this house: [1997, 3235, 5, 2.0, 1.16, 1]
$482,110.41, took 14 ms

Using model LosAngeles_CA.tar.gz to predict price of this house: [1994, 3382, 4, 1.0, 0.6, 3]
$503,948.69, took 17 ms

Using model LosAngeles_CA.tar.gz to predict price of this house: [1978, 2604, 2, 2.0, 1.27, 3]
$310,765.97, took 14 ms

Using model Houston_TX.tar.gz to predict price of t

### Dynamically deploy another model
Here we demonstrate the power of dynamic loading of new models. We purposely did not copy the first model when deploying models earlier. Now we deploy an additional model and can immediately invoke it through the multi-model endpoint. As with the earlier models, the first invocation to the new model takes longer, as the endpoint takes time to download the model and load it into memory.

In [28]:
# add another model to the endpoint and exercise it
deploy_artifacts_to_mme(training_jobs[0])

Copying NewYork_NY model
   from: DEMO_MME_XGBOOST/model_artifacts/NewYork_NY/xgb-NewYork-NY-2020-01-22-06-41-05-962/output/model.tar.gz
     to: DEMO_MME_XGBOOST/multi_model_artifacts/NewYork_NY.tar.gz...


'DEMO_MME_XGBOOST/multi_model_artifacts/NewYork_NY.tar.gz'

### Invoke the newly deployed model
Exercise the newly deployed model without the need for any endpoint update or restart.

In [29]:
print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls $model_url

Here are the models that the endpoint has at its disposal:
2020-01-22 06:46:13      11590 Chicago_IL.tar.gz
2020-01-22 06:46:13      11235 Houston_TX.tar.gz
2020-01-22 06:46:13      11946 LosAngeles_CA.tar.gz
2020-01-22 06:52:48      11535 NewYork_NY.tar.gz


In [30]:
model_name = LOCATIONS[0]
full_model_name = '{}.tar.gz'.format(model_name)
for i in range(5):
    features = gen_random_house()
    predict_one_house_value(gen_random_house()[1:], full_model_name)

Using model NewYork_NY.tar.gz to predict price of this house: [1995, 1992, 3, 3.0, 1.11, 1]
$275,930.69, took 778 ms

Using model NewYork_NY.tar.gz to predict price of this house: [2004, 2915, 3, 2.5, 1.02, 1]
$450,495.28, took 17 ms

Using model NewYork_NY.tar.gz to predict price of this house: [1985, 2687, 3, 3.0, 0.9, 3]
$364,312.75, took 15 ms

Using model NewYork_NY.tar.gz to predict price of this house: [1997, 3712, 5, 1.0, 0.9, 0]
$522,601.09, took 17 ms

Using model NewYork_NY.tar.gz to predict price of this house: [1995, 4212, 2, 2.5, 1.06, 1]
$617,013.75, took 17 ms



### Updating a model
To update a model, you would follow the same approach as above and add it as a new model. For example, if you have retrained the `NewYork_NY.tar.gz` model and wanted to start invoking it, you would upload the updated model artifacts behind the S3 prefix with a new name such as `NewYork_NY_v2.tar.gz`, and then change the `TargetModel` field to invoke `NewYork_NY_v2.tar.gz` instead of `NewYork_NY.tar.gz`. You do not want to overwrite the model artifacts in Amazon S3, because the old version of the model might still be loaded in the containers or on the storage volume of the instances on the endpoint. Invocations to the new model could then invoke the old version of the model.

Alternatively, you could stop the endpoint and re-deploy a fresh set of models.

## Clean up
Here, to be sure we are not billed for endpoints we are no longer using, we clean up.

In [31]:
# shut down the endpoint
#sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '65e5c7ba-1851-4fc7-92c9-15a9e2c8a959',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '65e5c7ba-1851-4fc7-92c9-15a9e2c8a959',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 22 Jan 2020 06:52:49 GMT'},
  'RetryAttempts': 0}}

In [32]:
# and the endpoint config
#sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

{'ResponseMetadata': {'RequestId': 'b15af137-dd73-45b7-bd68-7821b6cd5703',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b15af137-dd73-45b7-bd68-7821b6cd5703',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 22 Jan 2020 06:52:49 GMT'},
  'RetryAttempts': 0}}

In [33]:
# delete model too
#sm_client.delete_model(ModelName=multi_model_name)

{'ResponseMetadata': {'RequestId': '9b3f3411-6f8d-4d3b-8b5c-2f38f0c499b4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9b3f3411-6f8d-4d3b-8b5c-2f38f0c499b4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 22 Jan 2020 06:52:49 GMT'},
  'RetryAttempts': 0}}